In [16]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType
from pyspark.sql import Window
from pyspark import SparkContext, SparkConf, SQLContext


spark = SparkSession.builder.appName('spark_csv').getOrCreate()

### Считать csv-файл из первого этапа и преобразовать данные этого файла в spark-dataframe

In [3]:
df = spark.read.csv(path ='phones.csv',header=True)

In [4]:
display(df.show(truncate=False))

+-------------------------------------------------+------+-------------------------+-----+------+
|title                                            |price |seller                   |ram  |memory|
+-------------------------------------------------+------+-------------------------+-----+------+
|Apple iPhone 12 Pro 256Gb Pacific Blue           |558875|Mobilion                 |6 Гб |256Гб |
|Xiaomi Poco X3 NFC 6/128Gb Cobalt Blue           |115100|i-Store                  |6 Гб |128Гб |
|Xiaomi Redmi Note 9 Pro 6/128Gb Glacier White    |110000|i-Store                  |6 Гб |128Гб |
|Xiaomi Redmi Note 9 Pro 6/64Gb Glacier White     |89480 |Great Choice             |6 Гб |64Гб  |
|Xiaomi Redmi Note 9 Pro 6/64Gb Interstellar Gray |89480 |Great Choice             |6 Гб |64Гб  |
|Xiaomi Redmi Note 9 Pro 6/64Gb Glacier White     |89634 |Tsmart.kz                |6 Гб |64Гб  |
|Xiaomi Redmi 7 3/32Gb Blue                       |52900 |mobitron                 |3 Гб |32Гб  |
|Samsung Galaxy S20 

None

In [5]:
split_col = F.split(df['title'], ' ')

#Разделить поле title на два отдельных – smartphone_brand и smartphone_model. 
df1 = df.withColumn('smartphone_brand', F.split(df['title'],' ').getItem(0))
df1 = df1.withColumn('smartphone_brand1', split_col.getItem(F.size(split_col)-1))
df1 = df1.withColumn('smartphone_brand2', split_col.getItem(F.size(split_col)-2))
df1 = df1.drop('seller','ram','memory')
df1 = df1.withColumn('smartphone_model',F.expr('substring(title,length(smartphone_brand)+2, \
length(title)-length(smartphone_brand)-length(smartphone_brand1)-length(smartphone_brand2)-3)')).drop('smartphone_brand2','smartphone_brand1')
# я не знал как сделать по-другому - ничего не нашел


#Найти средние стоимости моделей смартфонов. 
df1 = df1.withColumn("price", df1["price"].cast(IntegerType()))
df1 = df1.groupby('smartphone_brand','smartphone_model').agg(F.avg('price').alias('smartphone_price'))

#Каждой уникальной записи dataframe’а необходимо присвоить свой порядковый smartphone_id, начиная с 1
df1= df1.withColumn("new_column",F.lit("ABC"))
w = Window().partitionBy('new_column').orderBy(F.lit('A'))
df1 = df1.withColumn("smartphone_id", F.row_number().over(w)).drop("new_column")
df1 = df1.select('smartphone_id', 'smartphone_brand', 'smartphone_model', 'smartphone_price')
df1.show(10,False)





+-------------+----------------+----------------------------+-----------------+
|smartphone_id|smartphone_brand|smartphone_model            |smartphone_price |
+-------------+----------------+----------------------------+-----------------+
|1            |Xiaomi          |Poco X3 NFC 6/128Gb         |115100.0         |
|2            |Xiaomi          |Redmi Note 9 Pro 6/128Gb    |110000.0         |
|3            |Xiaomi          |Redmi 7                     |52900.0          |
|4            |Apple           |iPhone 7 32Gb               |137700.0         |
|5            |Xiaomi          |Redmi Note 9 Pro 6/64Gb     |89531.33333333333|
|6            |Samsung         |Galaxy S20 Ultra 5G 12/128Gb|343489.0         |
|7            |Apple           |iPhone 12 Pro 256Gb         |558875.0         |
+-------------+----------------+----------------------------+-----------------+



In [11]:
df1.write.mode('overwrite').parquet("phones.parquet")

In [33]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars ~/Downloads/postgresql-42.2.18.jar pyspark-shell'


In [34]:
url = "jdbc:postgresql://localhost:5432/beeline_db"
properties = {
    "driver": "~/Downloads/postgresql-42.2.18.jar",
    "user": "rinatmirzagalamov",
    
}

In [36]:
mode = 'overwrite'
df1.write.jdbc(url=url, table="smartphones", mode=mode, properties=properties)

Py4JJavaError: An error occurred while calling o160.jdbc.
: java.lang.ClassNotFoundException: ~/Downloads/postgresql-42.2.18.jar
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:471)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:589)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:522)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:45)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:99)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:99)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:99)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:194)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:1